In [1]:
import os
import traceback
import numpy as np
import time
#from cuda import cuda, nvrtc

from hyperbolicTSNE.util import find_last_embedding
from hyperbolicTSNE.visualization import plot_poincare, animate
from hyperbolicTSNE import load_data, Datasets, SequentialOptimizer, initialization, HyperbolicTSNE

data_home = "datasets"
log_path = "temp/poincaregpu/"  # path for saving embedding snapshots

only_animate = False
seed = 42
dataset = Datasets.PLANARIA  # the Datasets handler provides access to several data sets used throughout the repository
num_points = 20000  # we use a subset for demonstration purposes, full MNIST has N=70000
perp = 30  # we use a perplexity of 30 in this example

dataX, dataLabels, D, V, _ = load_data(
    dataset, 
    data_home=data_home, 
    random_state=seed, 
    to_return="X_labels_D_V",
    hd_params={"perplexity": perp}, 
    sample=num_points, 
    knn_method="hnswlib"  # we use an approximation of high-dimensional neighbors to speed up computations
)

print("Data loaded")

exaggeration_factor = 12  # Just like regular t-SNE, we use early exaggeration with a factor of 12
learning_rate = (dataX.shape[0] * 1) / (exaggeration_factor * 1000)  # We adjust the learning rate to the hyperbolic setting
ex_iterations = 250  # The embedder is to execute 250 iterations of early exaggeration, ...
main_iterations = 750  # ... followed by 750 iterations of non-exaggerated gradient descent.












# ============= RUNNING EXACT GPU =============







opt_config = dict(
    learning_rate_ex=learning_rate,  # learning rate during exaggeration
    learning_rate_main=learning_rate,  # learning rate main optimization 
    exaggeration=exaggeration_factor, 
    exaggeration_its=ex_iterations, 
    gradientDescent_its=main_iterations, 
    vanilla=False,  # if vanilla is set to true, regular gradient descent without any modifications is performed; for  vanilla set to false, the optimization makes use of momentum and gains
    momentum_ex=0.5,  # Set momentum during early exaggeration to 0.5
    momentum=0.8,  # Set momentum during non-exaggerated gradient descent to 0.8
    exact=False,  # To use the quad tree for acceleration (like Barnes-Hut in the Euclidean setting) or to evaluate the gradient exactly
    area_split=False,  # To build or not build the polar quad tree based on equal area splitting or - alternatively - on equal length splitting
    n_iter_check=10,  # Needed for early stopping criterion
    size_tol=0.999  # Size of the embedding to be used as early stopping criterion
)

opt_params = SequentialOptimizer.sequence_poincare(**opt_config)

print("Sequence defined")

# Start: configure logging
logging_dict = {
    "log_path": log_path
}
opt_params["logging_dict"] = logging_dict

log_path = opt_params["logging_dict"]["log_path"]
# Delete old log path
if os.path.exists(log_path) and not only_animate:
    import shutil
    shutil.rmtree(log_path)
# End: logging

# Compute an initial embedding of the data via PCA
X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)
print(X_embedded)

# Initialize the embedder
htsne = HyperbolicTSNE(
    init=X_embedded, 
    n_components=2, 
    metric="precomputed", 
    verbose=True, 
    opt_method=SequentialOptimizer, 
    opt_params=opt_params
)

X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)
print(X_embedded)

start_time = time.time()

try:
    hyperbolicEmbedding = htsne.fit_transform((D, V))
except ValueError:
    print("Error!")
    hyperbolicEmbedding = find_last_embedding(log_path)
    traceback.print_exc()

end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Create a rendering of the embedding and save it to a file
if not os.path.exists("results"):
    os.mkdir("results")
fig = plot_poincare(hyperbolicEmbedding, dataLabels)
fig.show()
fig.savefig(f"results/{dataset.name}-inexact.png")












# ============= RUNNING EXACT CPU =============






opt_config = dict(
    learning_rate_ex=learning_rate,  # learning rate during exaggeration
    learning_rate_main=learning_rate,  # learning rate main optimization 
    exaggeration=exaggeration_factor, 
    exaggeration_its=ex_iterations, 
    gradientDescent_its=main_iterations, 
    vanilla=False,  # if vanilla is set to true, regular gradient descent without any modifications is performed; for  vanilla set to false, the optimization makes use of momentum and gains
    momentum_ex=0.5,  # Set momentum during early exaggeration to 0.5
    momentum=0.8,  # Set momentum during non-exaggerated gradient descent to 0.8
    exact=True,  # To use the quad tree for acceleration (like Barnes-Hut in the Euclidean setting) or to evaluate the gradient exactly
    area_split=False,  # To build or not build the polar quad tree based on equal area splitting or - alternatively - on equal length splitting
    n_iter_check=10,  # Needed for early stopping criterion
    size_tol=0.999  # Size of the embedding to be used as early stopping criterion
)

opt_params = SequentialOptimizer.sequence_poincare(**opt_config)

print("Sequence defined")

log_path = "temp/poincarecpu/"  # path for saving embedding snapshots

# Start: configure logging
logging_dict = {
    "log_path": log_path
}
opt_params["logging_dict"] = logging_dict

log_path = opt_params["logging_dict"]["log_path"]
# Delete old log path
if os.path.exists(log_path) and not only_animate:
    import shutil
    shutil.rmtree(log_path)
# End: logging

# Compute an initial embedding of the data via PCA
X_embedded = initialization(
    n_samples=dataX.shape[0],
    n_components=2,
    X=dataX,
    random_state=seed,
    method="pca"
)

# Initialize the embedder
htsne = HyperbolicTSNE(
    init=X_embedded, 
    n_components=2, 
    metric="precomputed", 
    verbose=True, 
    opt_method=SequentialOptimizer, 
    opt_params=opt_params
)

start_time = time.time()

try:
    hyperbolicEmbedding = htsne.fit_transform((D, V))
except ValueError:
    print("Error!")
    hyperbolicEmbedding = find_last_embedding(log_path)
    traceback.print_exc()

end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

# Create a rendering of the embedding and save it to a file
if not os.path.exists("results"):
    os.mkdir("results")
fig = plot_poincare(hyperbolicEmbedding, dataLabels)
fig.show()
fig.savefig(f"results/{dataset.name}-exact.png")

Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.
Data loaded
Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.
Sequence defined
[[-3.83978186e-05  6.36707264e-05]
 [ 2.12960411e-04  2.11776161e-04]
 [ 1.64758967e-04  1.70639410e-04]
 ...
 [ 1.09413224e-04  3.22986780e-05]
 [-3.89825000e-05 -2.25648346e-05]
 [-9.27048768e-06 -5.83760266e-05]]
[[-3.83978186e-05  6.36707264e-05]
 [ 2.12960411e-04  2.11776161e-04]
 [ 1.64758967e-04  1.70639410e-04]
 ...
 [ 1.09413224e-04  3.22986780e-05]
 [-3.89825000e-05 -2.25648346e-05]
 [-9.27048768e-06 -5.83760266e-05]]
[HyperbolicTSNE] Received iterable as input. It should have len=2 and contain (D=None, V=None)
[hd_mat] Warning: There is nothing to do with given parameters. Returning given D and V
Running Gradient Descent, Verbosity: True
[gradient_descent] Warning!: because of logging, the cf will be computed at every iteration


Gradient Descent:   0%|          | 0/250 [00:00<?, ?it/s]

LaunchError: cuLaunchKernel failed: too many resources requested for launch

Exception ignored in: 'hyperbolicTSNE.hyperbolic_barnes_hut.tsne.uniform_grid_compute_gradient_gpu'
Traceback (most recent call last):
  File "hyperbolicTSNE\hyperbolic_barnes_hut\gpu.py", line 200, in hyperbolicTSNE.hyperbolic_barnes_hut.tsne.uniform_grid_compute_gradient_negative_gpu
    cuda_func(np.int32(start),
  File "c:\Users\Milan\miniconda3\envs\htsne\lib\site-packages\pycuda\driver.py", line 502, in function_call
    func._launch_kernel(grid, block, arg_buf, shared, None)
pycuda._driver.LaunchError: cuLaunchKernel failed: too many resources requested for launch
Gradient Descent error: 0.00000 grad_norm: 0.00000e+00:   0%|          | 1/250 [00:01<04:35,  1.10s/it]

LaunchError: cuLaunchKernel failed: too many resources requested for launch

...

Exception ignored in: 'hyperbolicTSNE.hyperbolic_barnes_hut.tsne.uniform_grid_compute_gradient_gpu'
Traceback (most recent call last):
  File "hyperbolicTSNE\hyperbolic_barnes_hut\gpu.py", line 200, in hyperbolicTSNE.hyperbolic_barnes_hut.tsne.uniform_grid_compute_gradient_negative_gpu
    cuda_func(np.int32(start),
  File "c:\Users\Milan\miniconda3\envs\htsne\lib\site-packages\pycuda\driver.py", line 502, in function_call
    func._launch_kernel(grid, block, arg_buf, shared, None)
pycuda._driver.LaunchError: cuLaunchKernel failed: too many resources requested for launch
Gradient Descent error: 0.00000 grad_norm: 0.00000e+00:   1%|          | 8/750 [00:00<01:00, 12.20it/s]

LaunchError: cuLaunchKernel failed: too many resources requested for launch

Exception ignored in: 'hyperbolicTSNE.hyperbolic_barnes_hut.tsne.uniform_grid_compute_gradient_gpu'
Traceback (most recent call last):
  File "hyperbolicTSNE\hyperbolic_barnes_hut\gpu.py", line 200, in hyperbolicTSNE.hyperbolic_barnes_hut.tsne.uniform_grid_compute_gradient_negative_gpu
    cuda_func(np.int32(start),
  File "c:\Users\Milan\miniconda3\envs\htsne\lib\site-packages\pycuda\driver.py", line 502, in function_call
    func._launch_kernel(grid, block, arg_buf, shared, None)
pycuda._driver.LaunchError: cuLaunchKernel failed: too many resources requested for launch
Gradient Descent error: 0.00000 grad_norm: 0.00000e+00:   1%|          | 9/750 [00:00<01:10, 10.49it/s]
C:\Users\Milan\AppData\Local\Temp\ipykernel_12024\1048187734.py:135: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


3
Execution time: 2.956752300262451 seconds
Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.
Sequence defined
[HyperbolicTSNE] Received iterable as input. It should have len=2 and contain (D=None, V=None)
[hd_mat] Warning: There is nothing to do with given parameters. Returning given D and V
Running Gradient Descent, Verbosity: True
[gradient_descent] Warning!: because of logging, the cf will be computed at every iteration


Gradient Descent error: 93.97985 grad_norm: 8.32096e-01:  16%|█▌        | 39/250 [00:12<01:07,  3.13it/s]